In [206]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

슝=3


In [207]:
# 포지셔널 인코딩 레이어

class PositionalEncoding(tf.keras.layers.Layer):

  def __init__(self, position, d_model):
    super(PositionalEncoding, self).__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)

  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles

  def positional_encoding(self, position, d_model):
    # 각도 배열 생성
    angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)

    # 배열의 짝수 인덱스에는 sin 함수 적용
    sines = tf.math.sin(angle_rads[:, 0::2])
    # 배열의 홀수 인덱스에는 cosine 함수 적용
    cosines = tf.math.cos(angle_rads[:, 1::2])

    # sin과 cosine이 교차되도록 재배열
    pos_encoding = tf.stack([sines, cosines], axis=0)
    pos_encoding = tf.transpose(pos_encoding,[1, 2, 0]) 
    pos_encoding = tf.reshape(pos_encoding, [position, d_model])

    pos_encoding = pos_encoding[tf.newaxis, ...]
    return tf.cast(pos_encoding, tf.float32)

  def call(self, inputs):
    return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

슝=3


In [208]:
# 스케일드 닷 프로덕트 어텐션 함수

def scaled_dot_product_attention(query, key, value, mask):
  # 어텐션 가중치는 Q와 K의 닷 프로덕트
  matmul_qk = tf.matmul(query, key, transpose_b=True)

  # 가중치를 정규화
  depth = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(depth)

  # 패딩에 마스크 추가
  if mask is not None:
    logits += (mask * -1e9)

  # softmax적용
  attention_weights = tf.nn.softmax(logits, axis=-1)

  # 최종 어텐션은 가중치와 V의 닷 프로덕트
  output = tf.matmul(attention_weights, value)
  return output

슝=3


In [209]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    self.dense = tf.keras.layers.Dense(units=d_model)

  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    # Q, K, V에 각각 Dense를 적용합니다
    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value)

    # 병렬 연산을 위한 머리를 여러 개 만듭니다
    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    # 스케일드 닷 프로덕트 어텐션 함수 사용
    scaled_attention = scaled_dot_product_attention(query, key, value, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    # 어텐션 연산 후에 각 결과를 다시 연결(concatenate)합니다
    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    # 최종 결과에도 Dense를 한 번 더 적용합니다
    outputs = self.dense(concat_attention)

    return outputs

슝=3


In [210]:
def create_padding_mask(x):
  mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  # (batch_size, 1, 1, sequence length)
  return mask[:, tf.newaxis, tf.newaxis, :]

슝=3


In [211]:
def create_look_ahead_mask(x):
  seq_len = tf.shape(x)[1]
  look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
  padding_mask = create_padding_mask(x)
  return tf.maximum(look_ahead_mask, padding_mask)

슝=3


In [212]:
# 인코더 하나의 레이어를 함수로 구현.
# 이 하나의 레이어 안에는 두 개의 서브 레이어가 존재합니다.
def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")

  # 패딩 마스크 사용
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 첫 번째 서브 레이어 : 멀티 헤드 어텐션 수행 (셀프 어텐션)
  attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })

  # 어텐션의 결과는 Dropout과 Layer Normalization이라는 훈련을 돕는 테크닉을 수행
  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
  attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

  # 두 번째 서브 레이어 : 2개의 완전연결층
  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 완전연결층의 결과는 Dropout과 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

슝=3


In [213]:
def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name="encoder"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")

  # 패딩 마스크 사용
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 임베딩 레이어
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

  # 포지셔널 인코딩
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  # num_layers만큼 쌓아올린 인코더의 층.
  for i in range(num_layers):
    outputs = encoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name="encoder_layer_{}".format(i),
    )([outputs, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

슝=3


In [84]:
# 디코더 하나의 레이어를 함수로 구현.
# 이 하나의 레이어 안에는 세 개의 서브 레이어가 존재합니다.
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name="look_ahead_mask")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  # 첫 번째 서브 레이어 : 멀티 헤드 어텐션 수행 (셀프 어텐션)
  attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': look_ahead_mask
      })

  # 멀티 헤드 어텐션의 결과는 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
  attention1 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

  # 두 번째 서브 레이어 : 마스크드 멀티 헤드 어텐션 수행 (인코더-디코더 어텐션)
  attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1,
          'key': enc_outputs,
          'value': enc_outputs,
          'mask': padding_mask
      })

  # 마스크드 멀티 헤드 어텐션의 결과는
  # Dropout과 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
  attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
  attention2 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

  # 세 번째 서브 레이어 : 2개의 완전연결층
  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention2)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 완전연결층의 결과는 Dropout과 LayerNormalization 수행
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

슝=3


In [85]:
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name='decoder'):
  inputs = tf.keras.Input(shape=(None,), name='inputs')
  enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')

  # 패딩 마스크
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')
  
  # 임베딩 레이어
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

  # 포지셔널 인코딩
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

  # Dropout이라는 훈련을 돕는 테크닉을 수행
  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  for i in range(num_layers):
    outputs = decoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name='decoder_layer_{}'.format(i),
    )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

슝=3


데이터의 출처입니다.

https://github.com/songys/Chatbot_data/blob/master/ChatbotData.csv

In [219]:
data_row = pd.read_csv('ChatbotData.csv')

In [87]:
data = data_row.copy()

In [88]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11823 entries, 0 to 11822
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Q       11823 non-null  object
 1   A       11823 non-null  object
 2   label   11823 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 277.2+ KB


In [89]:
data

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [90]:
data.dropna(inplace=True) # 결측치 제거

In [91]:
data.drop_duplicates(inplace=True) # 중복 제거

In [92]:
data

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [93]:
import re

def clean_text(text):
    text = re.sub(r'[^\w\s가-힣]', '', text)  # 특수문자 제거 (영어 포함 가능)
    text = re.sub(r'\s+', ' ', text).strip()  # 공백 제거
    return text

data['Q'] = data['Q'].apply(clean_text)
data['A'] = data['A'].apply(clean_text)

In [94]:
data

,Q,A,label
0,12시 땡,하루가 또 가네요,0
1,1지망 학교 떨어졌어,위로해 드립니다,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠,0
4,PPL 심하네,눈살이 찌푸려지죠,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임,티가 나니까 눈치가 보이는 거죠,2
11819,훔쳐보는 것도 눈치 보임,훔쳐보는 거 티나나봐요,2
11820,흑기사 해주는 짝남,설렜겠어요,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까,잘 헤어질 수 있는 사이 여부인 거 같아요,2


In [95]:
# 토크나이징 

questions = data['Q'].tolist()
answers = data['A'].tolist()

tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    questions + answers,
    target_vocab_size = len(data)
#     target_vocab_size=2**13
)

# vocab 확인
print("Vocabulary size:", tokenizer.vocab_size)

Vocabulary size: 10125


In [96]:
from collections import Counter

threshold = 4

# 전체 문장 리스트 (이미 정제된 질문 + 답변 리스트)
corpus = questions + answers

# 서브워드 토큰 등장 횟수 계산
token_counter = Counter()
for sentence in corpus:
    token_ids = tokenizer.encode(sentence)
    token_counter.update(token_ids)

# 통계 계산
total_cnt = tokenizer.vocab_size
total_freq = sum(token_counter.values())

rare_cnt = 0
rare_freq = 0

for token_id, freq in token_counter.items():
    if freq < threshold:
        rare_cnt += 1
        rare_freq += freq

print('서브워드 집합(vocabulary)의 크기 :', total_cnt)
print('등장 빈도가 %d번 미만인 희귀 서브워드 수: %d' % (threshold, rare_cnt))
print('희귀 서브워드 제외 시 단어 집합의 크기 : %d' % (total_cnt - rare_cnt))
print('서브워드 집합에서 희귀 서브워드 비율: %.2f%%' % ((rare_cnt / total_cnt) * 100))
print('전체 등장 빈도에서 희귀 서브워드 비율: %.2f%%' % ((rare_freq / total_freq) * 100))


서브워드 집합(vocabulary)의 크기 : 10125
등장 빈도가 4번 미만인 희귀 서브워드 수: 2952
희귀 서브워드 제외 시 단어 집합의 크기 : 7173
서브워드 집합에서 희귀 서브워드 비율: 29.16%
전체 등장 빈도에서 희귀 서브워드 비율: 7.90%


희귀 서브워드를 날리는 것보다 사용하기로 선택

전체 등장 빈도에서 8%나 차지함

In [97]:
questions = data['Q'].tolist()
answers = data['A'].tolist()

tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    questions + answers,
    target_vocab_size = len(data)
)

# vocab 확인
print("Vocabulary size:", tokenizer.vocab_size)

Vocabulary size: 10125


In [98]:
# 시작 토큰과 종료 토큰에 고유한 정수를 부여합니다.
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]
print('START_TOKEN의 번호 :' ,[tokenizer.vocab_size])
print('END_TOKEN의 번호 :' ,[tokenizer.vocab_size + 1])
# 시작 토큰과 종료 토큰을 고려하여 +2를 하여 단어장의 크기를 산정합니다.
VOCAB_SIZE = tokenizer.vocab_size + 2
print(VOCAB_SIZE)

슝=3
START_TOKEN의 번호 : [10125]
END_TOKEN의 번호 : [10126]
10127


In [99]:
# 임의의 22번째 샘플에 대해서 정수 인코딩 작업을 수행.
# 각 토큰을 고유한 정수로 변환
print('정수 인코딩 후의 21번째 질문 샘플: {}'.format(tokenizer.encode(questions[21])))
print('정수 인코딩 후의 21번째 답변 샘플: {}'.format(tokenizer.encode(answers[21])))

정수 인코딩 후의 21번째 질문 샘플: [9808, 2192, 3609]
정수 인코딩 후의 21번째 답변 샘플: [2064, 6557, 5, 5462, 123]


In [100]:
def encode_sentence(sentence):
    return START_TOKEN + tokenizer.encode(sentence) + END_TOKEN

# questions와 answers 각각 인코딩
questions_encoded = [encode_sentence(sentence) for sentence in questions]
answers_encoded = [encode_sentence(sentence) for sentence in answers]

# 예시 출력
print("원본 질문:", questions[0])
print("인코딩된 질문:", questions_encoded[0])
print("디코딩 확인:", tokenizer.decode(questions_encoded[0][1:-1]))

원본 질문: 12시 땡
인코딩된 질문: [10125, 6932, 3006, 4776, 10126]
디코딩 확인: 12시 땡


In [101]:
total_data_text = questions_encoded + answers_encoded
# 텍스트데이터 문장길이의 리스트를 생성한 후
num_tokens = [len(tokens) for tokens in total_data_text]
num_tokens = np.array(num_tokens)
# 문장길이의 평균값, 최대값, 표준편차를 계산해 본다.
print('문장길이 평균 : ', np.mean(num_tokens))
print('문장길이 최대 : ', np.max(num_tokens))
print('문장길이 표준편차 : ', np.std(num_tokens))

# 예를들어, 최대 길이를 (평균 + 2*표준편차)로 한다면,
max_tokens = np.mean(num_tokens) + 3 * np.std(num_tokens)
maxlen = int(max_tokens)
print('pad_sequences maxlen : ', maxlen)
print(f'전체 문장의 {np.sum(num_tokens < max_tokens) / len(num_tokens)}%가 maxlen 설정값 이내에 포함됩니다. ')

문장길이 평균 :  6.730694409202402
문장길이 최대 :  28
문장길이 표준편차 :  2.3165233036673962
pad_sequences maxlen :  13
전체 문장의 0.9868899602469763%가 maxlen 설정값 이내에 포함됩니다. 


In [102]:
# 샘플의 최대 허용 길이 또는 패딩 후의 최종 길이
MAX_LENGTH = 13
print(MAX_LENGTH)

13


In [103]:
# 정수 인코딩, 최대 길이를 초과하는 샘플 제거, 패딩
def tokenize_and_filter(inputs, outputs):
  tokenized_inputs, tokenized_outputs = [], []

  for (sentence1, sentence2) in zip(inputs, outputs):
    # 정수 인코딩 과정에서 시작 토큰과 종료 토큰을 추가
    sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
    sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN

    # 최대 길이 40 이하인 경우에만 데이터셋으로 허용
    if len(sentence1) <= MAX_LENGTH and len(sentence2) <= MAX_LENGTH:
      tokenized_inputs.append(sentence1)
      tokenized_outputs.append(sentence2)

  # 최대 길이 40으로 모든 데이터셋을 패딩
  tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
  tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_outputs, maxlen=MAX_LENGTH, padding='post')

  return tokenized_inputs, tokenized_outputs

슝=3


In [104]:
questions, answers = tokenize_and_filter(questions, answers)
print('단어장의 크기 :',(VOCAB_SIZE))
print('필터링 후의 질문 샘플 개수: {}'.format(len(questions)))
print('필터링 후의 답변 샘플 개수: {}'.format(len(answers)))

단어장의 크기 : 10127
필터링 후의 질문 샘플 개수: 11516
필터링 후의 답변 샘플 개수: 11516


In [105]:
BATCH_SIZE = 64
BUFFER_SIZE = 20000

# 디코더는 이전의 target을 다음의 input으로 사용합니다.
# 이에 따라 outputs에서는 START_TOKEN을 제거하겠습니다.
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': questions,
        'dec_inputs': answers[:, :-1]
    },
    {
        'outputs': answers[:, 1:]
    },
))

# 배치 데이터셋을 생성
dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

슝=3


In [106]:
def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                name="transformer"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")
  dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

  # 인코더에서 패딩을 위한 마스크
  enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)

  # 디코더에서 미래의 토큰을 마스크 하기 위해서 사용합니다.
  # 내부적으로 패딩 마스크도 포함되어져 있습니다.
  look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask,
      output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)

  # 두 번째 어텐션 블록에서 인코더의 벡터들을 마스킹
  # 디코더에서 패딩을 위한 마스크
  dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)

  # 인코더
  enc_outputs = encoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
  )(inputs=[inputs, enc_padding_mask])

  # 디코더
  dec_outputs = decoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
  )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

  # 완전연결층
  outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

  return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)
print("슝=3")

슝=3


In [130]:
tf.keras.backend.clear_session()

# 하이퍼파라미터
NUM_LAYERS = 6 # 인코더와 디코더의 층의 개수
D_MODEL = 256 # 인코더와 디코더 내부의 입, 출력의 고정 차원
NUM_HEADS = 8 # 멀티 헤드 어텐션에서의 헤드 수
UNITS = 512 # 피드 포워드 신경망의 은닉층의 크기
DROPOUT = 0.1 # 드롭아웃의 비율

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

model.summary()

Model: "transformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, None)]       0                                            
__________________________________________________________________________________________________
dec_inputs (InputLayer)         [(None, None)]       0                                            
__________________________________________________________________________________________________
enc_padding_mask (Lambda)       (None, 1, 1, None)   0           inputs[0][0]                     
__________________________________________________________________________________________________
encoder (Functional)            (None, None, 256)    5755136     inputs[0][0]                     
                                                                 enc_padding_mask[0][0] 

In [131]:
def loss_function(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))

  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)

  mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
  loss = tf.multiply(loss, mask)

  return tf.reduce_mean(loss)
print("슝=3")

슝=3


In [132]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps**-1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)
print("슝=3")

슝=3


In [133]:
learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])
print("슝=3")

슝=3


In [134]:
EPOCHS = 50
model.fit(dataset, epochs=EPOCHS, verbose=1)

Epoch 1/50
180/180 [==============================] - 27s 68ms/step - loss: 3.8804 - accuracy: 0.0737
Epoch 2/50
180/180 [==============================] - 12s 67ms/step - loss: 3.3662 - accuracy: 0.0833
Epoch 3/50
180/180 [==============================] - 12s 67ms/step - loss: 3.0636 - accuracy: 0.0835
Epoch 4/50
180/180 [==============================] - 12s 67ms/step - loss: 2.8927 - accuracy: 0.0863
Epoch 5/50
180/180 [==============================] - 12s 67ms/step - loss: 2.7845 - accuracy: 0.0911
Epoch 6/50
180/180 [==============================] - 12s 67ms/step - loss: 2.7043 - accuracy: 0.0949
Epoch 7/50
180/180 [==============================] - 12s 67ms/step - loss: 2.6131 - accuracy: 0.1010
Epoch 8/50
180/180 [==============================] - 12s 67ms/step - loss: 2.5163 - accuracy: 0.1062
Epoch 9/50
180/180 [==============================] - 12s 66ms/step - loss: 2.4182 - accuracy: 0.1113
Epoch 10/50
180/180 [==============================] - 12s 67ms/step - loss: 2.309

In [136]:
import tensorflow as tf

# 예측 함수
def predict_answer(question, tokenizer, model, max_length=13):
    # 질문을 토큰화
    question_encoded = tokenizer.encode(question)
    question_padded = tf.keras.preprocessing.sequence.pad_sequences([question_encoded], maxlen=max_length, padding='post')

    # 디코더 입력은 시작 토큰으로 초기화
    start_token = tokenizer.vocab_size  # 시작 토큰 <START_TOKEN>
    dec_input = tf.constant([[start_token]], dtype=tf.int64)  # <START_TOKEN> 입력

    # 예측 시작
    for _ in range(max_length - 1):  # 최대 길이 (13까지 예측)
        # 예측을 얻는다 (모델에 입력)
        prediction = model([question_padded, dec_input])

        # 예측된 값 중 가장 확률이 높은 토큰을 선택
        predicted_token = tf.argmax(prediction, axis=-1)[:, -1].numpy()

        # 종료 토큰 <END_TOKEN>이 나오면 예측을 멈춘다
        if predicted_token == tokenizer.vocab_size + 1:  # <END_TOKEN>
            break

        # 예측된 토큰을 [1, 1] 형태로 변환하여 디코더 입력에 추가
        predicted_token = tf.reshape(predicted_token, (1, 1))  # 차원 맞추기
        predicted_token = tf.cast(predicted_token, tf.int64)  # 데이터 타입을 int64로 맞추기
        dec_input = tf.concat([dec_input, predicted_token], axis=-1)

    # 예측된 토큰을 디코딩하여 답변 생성
    predicted_answer = tokenizer.decode(dec_input.numpy()[0][1:])  # 시작 토큰 제외
    return predicted_answer

In [226]:
data_row['Q'][data_row['Q'].str.contains('사귀')]

265                        괜찮은 사람인데 사귀긴 싫어
488                          나 몰래 사귀는 거 같애
1350                           둘이 사귀는 거 같애
1507                        만나면 좋은데 사귀긴 싫어
2257                                사귀고 싶어
                       ...                
11503    짝녀가 사귀던 사람이랑 헤어졌는데 좋아한다고 고백해도 되나.
11692              친구도 아니고 사귀는 사이도 아닌데 뭐지?
11718               친한 친구의 구남친이랑 사귀어도 될까요?
11719                친한 친구의 구여친이랑 사귀어도 되나.
11739               클럽에서 만나서 사귀는거 어떻게 생각해?
Name: Q, Length: 70, dtype: object

In [142]:
# 예시로 질문 넣기
input_question = "마 나랑 사귀자 내가 잘해줄게"
answer = predict_answer(input_question, tokenizer, model)

# 결과 출력
print("질문:", input_question)
print("예측된 답변:", answer)


질문: 마 나랑 사귀자 내가 잘해줄게
예측된 답변: 자신을 더 사랑해주세요


In [202]:
# 예시로 질문 넣기
input_question = "나랑 사귀자 내가 잘해줄게"
answer = predict_answer(input_question, tokenizer, model)

# 결과 출력
print("질문:", input_question)
print("예측된 답변:", answer)

질문: 나랑 사귀자 내가 잘해줄게
예측된 답변: 이제 같은 실수 안 하면 돼요


In [147]:
# 예시로 질문 넣기
input_question = "자살 생각이 들어.."
answer = predict_answer(input_question, tokenizer, model)

# 결과 출력
print("질문:", input_question)
print("예측된 답변:", answer)


질문: 자살 생각이 들어..
예측된 답변: 마음 먹은 것만으로도 절반을 해낸 거예요


In [154]:
# 예시로 질문 넣기
input_question = "너 미쳤어? 나보고 자살하라는거지?"
answer = predict_answer(input_question, tokenizer, model)

# 결과 출력
print("질문:", input_question)
print("예측된 답변:", answer)


질문: 너 미쳤어? 나보고 자살하라는거지?
예측된 답변: 마음이 복잡한가봐요


In [215]:
# 예시로 질문 넣기
input_question = "인생이란 뭘까"
answer = predict_answer(input_question, tokenizer, model)

# 결과 출력
print("질문:", input_question)
print("예측된 답변:", answer)


질문: 인생이란 뭘까
예측된 답변: 스트레스 받지 마세요


In [216]:
# 예시로 질문 넣기
input_question = "피곤하네"
answer = predict_answer(input_question, tokenizer, model)

# 결과 출력
print("질문:", input_question)
print("예측된 답변:", answer)


질문: 피곤하네
예측된 답변: 아무래도 그렇겠죠


1. train valid 나눠서 일반화 성능을 올려보자

2. 인코더 디코더 레이어를 낮추고 입출력 차원, dense 차원을 낮춰보자

In [227]:
import tensorflow as tf

# 전체 데이터셋을 80% train, 20% validation으로 나누기
TRAIN_RATIO = 0.8
VAL_RATIO = 0.2

# 데이터셋 크기
dataset_size = len(questions)

# 훈련 데이터와 검증 데이터의 인덱스 크기
train_size = int(TRAIN_RATIO * dataset_size)
val_size = dataset_size - train_size

# 데이터셋을 shuffle한 뒤, 나누기
dataset = tf.data.Dataset.from_tensor_slices(( 
    {
        'inputs': questions,
        'dec_inputs': answers[:, :-1]
    },
    {
        'outputs': answers[:, 1:]
    },
))

# 데이터셋을 섞고 배치처리
dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)

# 전체 데이터셋을 80%는 훈련용, 20%는 검증용으로 분할
train_dataset = dataset.take(train_size // BATCH_SIZE)
val_dataset = dataset.skip(train_size // BATCH_SIZE)

# 각 데이터셋을 미리 처리하기 위한 Prefetch
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)
val_dataset = val_dataset.prefetch(tf.data.experimental.AUTOTUNE)

# 데이터셋 확인
print(f'Training dataset size: {train_size}')
print(f'Validation dataset size: {val_size}')


Training dataset size: 9212
Validation dataset size: 2304


In [228]:
tf.keras.backend.clear_session()

# 하이퍼파라미터
NUM_LAYERS = 4 # 인코더와 디코더의 층의 개수
D_MODEL = 128 # 인코더와 디코더 내부의 입, 출력의 고정 차원
NUM_HEADS = 4 # 멀티 헤드 어텐션에서의 헤드 수
UNITS = 256 # 피드 포워드 신경망의 은닉층의 크기
DROPOUT = 0.1 # 드롭아웃의 비율

model2 = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

model2.summary()

Model: "transformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, None)]       0                                            
__________________________________________________________________________________________________
dec_inputs (InputLayer)         [(None, None)]       0                                            
__________________________________________________________________________________________________
enc_padding_mask (Lambda)       (None, 1, 1, None)   0           inputs[0][0]                     
__________________________________________________________________________________________________
encoder (Functional)            (None, None, 128)    1826176     inputs[0][0]                     
                                                                 enc_padding_mask[0][0] 

In [229]:
EPOCHS = 100  # 에포크 수
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',        # 'val_loss'를 모니터링
    patience=3,                # 3 에포크 동안 성능 향상이 없으면 학습 중단
    restore_best_weights=True  # 가장 좋은 모델 가중치를 복원
)

# 모델 컴파일
model2.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

# 모델 학습
history = model2.fit(
    train_dataset,  # 훈련 데이터
    epochs=EPOCHS,  # 에포크 수
    validation_data=val_dataset,  # 검증 데이터
    callbacks=[early_stopping],  # 조기 종료 콜백
    verbose=1
)

Epoch 1/100
143/143 [==============================] - 19s 66ms/step - loss: 3.3279 - accuracy: 0.0827 - val_loss: 3.0944 - val_accuracy: 0.0833
Epoch 2/100
143/143 [==============================] - 7s 52ms/step - loss: 2.9764 - accuracy: 0.0843 - val_loss: 2.8496 - val_accuracy: 0.0877
Epoch 3/100
143/143 [==============================] - 7s 52ms/step - loss: 2.7990 - accuracy: 0.0887 - val_loss: 2.6976 - val_accuracy: 0.0915
Epoch 4/100
143/143 [==============================] - 7s 52ms/step - loss: 2.7184 - accuracy: 0.0938 - val_loss: 2.6363 - val_accuracy: 0.0976
Epoch 5/100
143/143 [==============================] - 8s 53ms/step - loss: 2.6365 - accuracy: 0.0987 - val_loss: 2.5491 - val_accuracy: 0.1035
Epoch 6/100
143/143 [==============================] - 8s 53ms/step - loss: 2.5311 - accuracy: 0.1042 - val_loss: 2.4387 - val_accuracy: 0.1085
Epoch 7/100
143/143 [==============================] - 7s 52ms/step - loss: 2.4377 - accuracy: 0.1101 - val_loss: 2.3097 - val_accuracy

In [230]:
# 예시로 질문 넣기
input_question = "결혼하면 좋아?"
answer = predict_answer(input_question, tokenizer, model2)

# 결과 출력
print("질문:", input_question)
print("예측된 답변:", answer)


질문: 결혼하면 좋아?
예측된 답변: 해봐요


In [241]:
# 예시로 질문 넣기
input_question = "너도 힘들지? 사람들 답변해주느라"
answer = predict_answer(input_question, tokenizer, model)

# 결과 출력
print("질문:", input_question)
print("예측된 답변:", answer)

질문: 너도 힘들지? 사람들 답변해주느라
예측된 답변: 술 한잔으로 잊혀질 수 있다면 얼마나 좋을까요


In [247]:
# 예시로 질문 넣기
input_question = "내가 좋아하는 마음이 다른 사람에게 상처가 되면 어떡해"
answer = predict_answer(input_question, tokenizer, model)

# 결과 출력
print("질문:", input_question)
print("예측된 답변:", answer)

질문: 내가 좋아하는 마음이 다른 사람에게 상처가 되면 어떡해
예측된 답변: 그 사람도 설렐 거예요


In [231]:
# 예시로 질문 넣기
input_question = "자살 생각이 들어.."
answer = predict_answer(input_question, tokenizer, model2)

# 결과 출력
print("질문:", input_question)
print("예측된 답변:", answer)


질문: 자살 생각이 들어..
예측된 답변: 정말 그래요 하나같이 공감되곤 하죠


In [232]:
# 예시로 질문 넣기
input_question = "나랑 사귀자 내가 잘해줄게"
answer = predict_answer(input_question, tokenizer, model)

# 결과 출력
print("질문:", input_question)
print("예측된 답변:", answer)

질문: 나랑 사귀자 내가 잘해줄게
예측된 답변: 이제 같은 실수 안 하면 돼요


In [233]:
# 예시로 질문 넣기
input_question = "인생이란 뭘까"
answer = predict_answer(input_question, tokenizer, model)

# 결과 출력
print("질문:", input_question)
print("예측된 답변:", answer)


질문: 인생이란 뭘까
예측된 답변: 스트레스 받지 마세요


In [234]:
# 예시로 질문 넣기
input_question = "피곤하네"
answer = predict_answer(input_question, tokenizer, model)

# 결과 출력
print("질문:", input_question)
print("예측된 답변:", answer)

질문: 피곤하네
예측된 답변: 아무래도 그렇겠죠


### 회고
실험을 통해 다음과 같은 사실을 알게되었습니다.

- train, valid 나눠서 돌리니까 일반화 성능이 더 올라감

- 데이터셋 규모에 비해 모델이 무거우면 과적합이 빠르게 진행됨